In [1]:
import time
from functools import lru_cache
from pathlib import Path
from time import sleep
from typing import Any, Dict, Optional
from urllib.parse import urlparse

import awswrangler as wr
import boto3
import datahub.emitter.mce_builder as builder
import pandas as pd
from datahub.ingestion.graph.client import DatahubClientConfig, DataHubGraph
from ds_dqv_tool import dqv_check, dqv_check_yaml
from ds_dqv_tool.recipes import condition_description_map, metric_description_map
from jinja2 import DebugUndefined, Template

graph = DataHubGraph(config=DatahubClientConfig(server="http://localhost:8091"))

# [methods for methods in graph.__dir__() if "assertion" in methods]


@lru_cache
def get_athena_table_dataset_urn(catalog: str, database: str, table: str, region: str) -> str:
    """e.g. urn:li:dataset:(urn:li:dataPlatform:hive,/iceberg/yellow_rides_hourly_actuals,PROD)"""
    session = boto3.Session(profile_name="sandbox")
    athena_client = session.client("athena", region_name=region)
    table_metadata = athena_client.get_table_metadata(CatalogName=catalog, DatabaseName=database, TableName=table)

    # Dataset has also its' physical location which we can add in symlink facet.
    s3_location = table_metadata["TableMetadata"]["Parameters"]["location"]
    parsed_path = urlparse(s3_location)

    return builder.make_dataset_urn(
        platform="hive",
        name=parsed_path.path,
    )

In [2]:
from datahub.emitter.serialization_helper import pre_json_transform


def make_assertion_urn(dataset_urn: str, assertion_name: str) -> str:
    return builder.make_assertion_urn(
        builder.datahub_guid(
            pre_json_transform(
                # these key-val pairs are essentially hashed; we want to choose pairs
                # that make the assertions unique (example: https://github.com/datahub-project/datahub/blob/d2d9d36987f20a9f7d6c973073d1404edf33e667/metadata-ingestion-modules/gx-plugin/src/datahub_gx_plugin/action.py#L277-L289)
                {
                    "platform": "pattern-ds-dqv",
                    # bad name since assertions and datasets have a many-to-many relationship
                    "dataset_urn": dataset_urn,
                    "assertion_name": assertion_name,
                }
            )
        )
    )

In [3]:
def substitute_map_into_string(string: str, values: dict[str, Any]) -> str:
    """
    Format a string using a dictionary with Jinja2 templating.

    :param string: The template string containing placeholders
    :param values: A dictionary of values to substitute into the template
    """
    template = Template(string, undefined=DebugUndefined)
    return template.render(values)


def query_pandas_from_athena(
    sql_query: str,
    glue_database: str,
    datalake_s3_bucket: str,
    # job_name: str,
    s3_output_location: Optional[str] = None,
    ctx: Optional[Dict[str, Any]] = None,
) -> pd.DataFrame:
    # Validate job_name format
    # if not _is_valid_snake_case_identifier(job_name):
    #     raise ValueError(f"job_name must be a valid lower snake case identifier. Got: {job_name}")

    # Apply Jinja2 templating if context is provided
    if ctx is not None:
        sql_query = substitute_map_into_string(sql_query, ctx)

    session = boto3.Session(profile_name="sandbox", region_name="us-east-1")
    # region_name = session.region_name or "us-east-1"
    if s3_output_location is None:
        s3_output_location = f"s3://{datalake_s3_bucket}/athena-results"  # /{job_name}"

    # Emit OpenLineage START event for SQL query
    # query_run_id = str(generate_new_uuid())
    # emit_openlineage_start_event(job_name, sql_query, query_run_id)

    # Execute query using AWS Data Wrangler
    df = wr.athena.read_sql_query(
        sql=sql_query,
        database=glue_database,
        s3_output=s3_output_location,
        boto3_session=session,
    )

    return df

In [4]:
THIS_DIR = Path("metaflow")
SQL_DIR = THIS_DIR / "sql"
sql_path = SQL_DIR / "prepare_training_data.sql"
glue_database = "nyc_taxi"
datalake_s3_bucket = "airflow-metaflow-6721"
as_of_datetime = "2025-06-01 00:00:00.000"
lookback_days = 30


querys = {
    "raw_weather": """SELECT unique_row_id, filename, region_type, region_code, region_name, year, month, meteorological_element, day_01, day_02, day_03, day_04, day_05, day_06, day_07, day_08, day_09, day_10, day_11, day_12, day_13, day_14, day_15, day_16, day_17, day_18, day_19, day_20, day_21, day_22, day_23, day_24, day_25, day_26, day_27, day_28, day_29, day_30, day_31 FROM nyc_taxi.raw_weather""",
    "yellow_rides_hourly_forecast": """SELECT * from nyc_taxi.yellow_rides_hourly_forecast""",
    "yellow_rides_hourly_actuals": """SELECT year , month , day , hour , pulocationid , total_rides from nyc_taxi.yellow_rides_hourly_actuals""",
    "raw_yellow": """SELECT unique_row_id , filename , vendorid , passenger_count , trip_distance , ratecodeid , store_and_fwd_flag , pulocationid , dolocationid , payment_type , fare_amount , extra , mta_tax , tip_amount , tolls_amount , improvement_surcharge , total_amount , congestion_surcharge , cbd_congestion_fee , airport_fee from nyc_taxi.raw_yellow""",
}


In [5]:
datasets = [
    "yellow_rides_hourly_forecast",
    "yellow_rides_hourly_actuals",
    "raw_weather",
    "raw_yellow",
]

In [6]:
entity_urn = get_athena_table_dataset_urn(
    catalog="AwsDataCatalog",
    database="nyc_taxi",
    table="raw_weather",
    region="us-east-1",
)

dataset_name_to_urn = {
    table: get_athena_table_dataset_urn(
        catalog="AwsDataCatalog",
        database="nyc_taxi",
        table=table,
        region="us-east-1",
    )
    for table in datasets
}


assertion_urn = make_assertion_urn(
    dataset_urn=entity_urn,
    assertion_name="test_assertion",
)

In [7]:
dfs = {}
for table in datasets:
    print(table)
    dfs[table] = query_pandas_from_athena(
        sql_query=querys[table],
        glue_database=glue_database,
        datalake_s3_bucket=datalake_s3_bucket,
    )


yellow_rides_hourly_forecast
yellow_rides_hourly_actuals
raw_weather
raw_yellow


In [ ]:
def datahub_upsert_assertion(
    entity_urn,
    assertion_urn,
    status,
    checks_description,
    properties,
):
    graph.upsert_custom_assertion(
        urn=assertion_urn,
        entity_urn=entity_urn,
        type="DQV",  # This categorizes your assertion in DataHub
        description=checks_description,
        # platform_urn="urn:li:dataPlatform:great-expectations", # OR you can provide 'platformName="My Custom Platform"'
        platform_name="metaflow",
        # external_url="https://my-monitoring-tool.com/result-for-this-assertion",  # Optional: link to monitoring tool
    )


def datahub_update_assertion_result(
    entity_urn,
    assertion_urn,
    status,
    checks_description,
    properties,
):
    graph.report_assertion_result(
        urn=assertion_urn,
        timestamp_millis=int(time.time() * 1000),
        type=status,
        properties=properties,
    )

In [18]:
def log_dqv_report_datahub(dqv_results, dataset_name_to_urn):
    for status in ["passed", "failed"]:
        for result in dqv_results[status]:
            for column, metrics in result["checks"].items():
                for metric, conditions in metrics.items():
                    metric_desc = metric_description_map.get(metric, metric)
                    for condition_tuple in conditions:
                        condition = condition_tuple["condition"]
                        value = condition_tuple["value"]
                        actual = condition_tuple["calculated_value"]
                        cond_desc = condition_description_map.get(condition, condition)
                        properties = [
                            {"key": "column", "value": column},
                            {"key": "metric", "value": metric},
                            {"key": "condition", "value": condition},
                            {"key": "expected", "value": value},
                            {"key": "actual", "value": float(actual)},
                        ]
                        datahub_upsert_assertion(
                            entity_urn=dataset_name_to_urn[result["dataset_name"]],
                            assertion_urn=make_assertion_urn(
                                dataset_urn=dataset_name_to_urn[result["dataset_name"]],
                                assertion_name=f"{column}_{metric}_{condition}_{value}",
                            ),
                            status="SUCCESS" if status == "passed" else "FAILURE",
                            checks_description=f"Column: {column} - {metric_desc} value {cond_desc} {value}",
                            properties=properties,
                        )
    sleep(8)
    for status in ["passed", "failed"]:
        for result in dqv_results[status]:
            for column, metrics in result["checks"].items():
                for metric, conditions in metrics.items():
                    metric_desc = metric_description_map.get(metric, metric)
                    for condition_tuple in conditions:
                        condition = condition_tuple["condition"]
                        value = condition_tuple["value"]
                        actual = condition_tuple["calculated_value"]
                        cond_desc = condition_description_map.get(condition, condition)
                        properties = [
                            {"key": "column", "value": column},
                            {"key": "metric", "value": metric},
                            {"key": "condition", "value": condition},
                            {"key": "expected", "value": value},
                            {"key": "actual", "value": float(actual)},
                        ]
                        datahub_update_assertion_result(
                            entity_urn=dataset_name_to_urn[result["dataset_name"]],
                            assertion_urn=make_assertion_urn(
                                dataset_urn=dataset_name_to_urn[result["dataset_name"]],
                                assertion_name=f"{column}_{metric}_{condition}_{value}",
                            ),
                            status="SUCCESS" if status == "passed" else "FAILURE",
                            checks_description=f"Column: {column} - {metric_desc} value {cond_desc} {value}",
                            properties=properties,
                        )

In [26]:
dqv_results = dqv_check_yaml("assertion_checks.yaml", datasets=dfs)
dqv_results

{'passed': [{'dataset_name': 'yellow_rides_hourly_forecast',
   'dataset_owner': {'data_team@company.com': 'U123456789'},
   'dataset_type': 'pandas',
   'checks': {'year': {'missing_percent': [{'condition': 'eq',
       'value': 0,
       'criticality': 'fail',
       'calculated_value': np.float64(0.0)}],
     'min': [{'condition': 'gte',
       'value': 2020,
       'criticality': 'fail',
       'calculated_value': np.int64(2025)}],
     'max': [{'condition': 'lte',
       'value': 2030,
       'criticality': 'fail',
       'calculated_value': np.int64(2025)}]},
    'month': {'missing_percent': [{'condition': 'eq',
       'value': 0,
       'criticality': 'fail',
       'calculated_value': np.float64(0.0)}],
     'min': [{'condition': 'gte',
       'value': 1,
       'criticality': 'fail',
       'calculated_value': np.int64(6)}],
     'max': [{'condition': 'lte',
       'value': 12,
       'criticality': 'fail',
       'calculated_value': np.int64(6)}]},
    'day': {'missing_percen

In [20]:
for name, df in dfs.items():
    print(name)
    display(df.describe().round(5))

yellow_rides_hourly_forecast


,year,month,day,hour,pulocationid,forecasted_total_rides
count,6240.0,6240.0,6240.0,6240.0,6240.0,6240.0
mean,2025.0,6.0,1.0,11.5,133.16923,18.25465
std,0.0,0.0,0.0,6.92274,77.05168,44.1934
min,2025.0,6.0,1.0,0.0,1.0,0.0
25%,2025.0,6.0,1.0,5.75,65.75,0.0
50%,2025.0,6.0,1.0,11.5,134.5,3.0
75%,2025.0,6.0,1.0,17.25,200.25,12.0
max,2025.0,6.0,1.0,23.0,265.0,501.0


yellow_rides_hourly_actuals


,year,month,day,hour,pulocationid,total_rides
count,240879.0,240879.0,240879.0,240879.0,240879.0,240879.0
mean,2025.0,4.53853,16.09038,11.91919,138.06191,35.0111
std,0.0,0.49854,8.67531,6.69065,76.27138,76.06195
min,2025.0,4.0,1.0,0.0,1.0,1.0
25%,2025.0,4.0,9.0,7.0,72.0,2.0
50%,2025.0,5.0,16.0,12.0,140.0,4.0
75%,2025.0,5.0,24.0,18.0,209.0,22.0
max,2025.0,6.0,31.0,23.0,265.0,966.0


raw_weather


,region_code,year,month,day_01,day_02,day_03,day_04,day_05,day_06,day_07,...,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31
count,37284.0,37284.0,37284.0,37284.00000,37284.00000,37284.00000,37284.00000,37284.00000,37284.00000,37284.00000,...,37284.00000,37284.00000,37284.00000,37284.00000,37284.00000,37284.00000,37284.00000,24856.00000,24856.00000,12428.00000
mean,24879.27036,2025.0,3.0,7.01274,5.36881,6.87188,7.73093,8.22360,7.16480,5.84627,...,5.10806,6.95070,8.60730,9.60103,9.87041,9.38722,9.76666,12.84167,12.87304,11.12627
std,13708.05752,0.0,0.81651,8.65796,9.16588,10.49438,10.81167,12.06425,11.72068,9.69948,...,10.09983,9.78653,9.32848,9.11860,9.26281,9.12876,9.16151,10.17275,10.17342,9.48659
min,1001.0,2025.0,2.0,-24.73000,-28.61000,-27.24000,-28.46000,-27.23000,-30.66000,-28.60000,...,-24.91000,-18.42000,-17.51000,-17.35000,-15.80000,-15.88000,-16.78000,-9.03000,-10.28000,-11.56000
25%,13045.0,2025.0,2.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.68000,0.33000,0.19000,0.30000,2.15000,2.69000,2.20000
50%,23211.0,2025.0,3.0,5.37000,2.74000,4.07000,5.67000,6.43000,4.64000,3.53000,...,2.01500,4.31000,6.81500,8.29000,8.77500,7.98000,8.73000,13.13000,13.72000,10.68500
75%,39007.0,2025.0,4.0,12.97000,11.16000,13.65000,14.80000,15.40000,13.00000,11.15000,...,12.03000,14.00000,15.67000,16.64000,17.01000,16.24000,16.91000,21.18250,20.80000,19.05250
max,48045.0,2025.0,4.0,41.87000,65.70000,85.99000,106.16000,136.83000,147.66000,137.77000,...,38.69000,64.14000,67.19000,92.55000,36.53000,73.70000,102.12000,93.26000,92.38000,61.14000


raw_yellow


,vendorid,passenger_count,trip_distance,ratecodeid,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,cbd_congestion_fee,airport_fee
count,8562398.0,6620492.0,8.562398e+06,6620492.0,8562398.0,8562398.0,8562398.0,8.562398e+06,8.562398e+06,8.562398e+06,8.562398e+06,8.562398e+06,8.562398e+06,8.562398e+06,6.620492e+06,8.562398e+06,6.620492e+06
mean,1.85981,1.30163,7.347980e+00,2.45004,161.90209,161.65919,0.95476,1.817578e+01,1.218550e+00,4.769300e-01,2.914490e+00,5.006200e-01,9.547500e-01,2.674798e+01,2.205110e+00,5.299600e-01,1.443000e-01
std,0.68048,0.73944,6.575107e+02,11.43111,66.23322,70.32285,0.74956,1.929194e+01,1.860500e+00,1.404200e-01,3.984240e+00,2.121950e+00,2.798500e-01,2.368200e+01,9.374900e-01,3.608100e-01,5.258700e-01
min,1.0,0.0,0.000000e+00,1.0,1.0,1.0,0.0,-1.777500e+03,-1.739000e+01,-5.000000e-01,-9.044000e+01,-1.481700e+02,-1.000000e+00,-1.793370e+03,-2.500000e+00,-7.500000e-01,-1.750000e+00
25%,2.0,1.0,1.050000e+00,1.0,116.0,112.0,1.0,8.600000e+00,0.000000e+00,5.000000e-01,0.000000e+00,0.000000e+00,1.000000e+00,1.573000e+01,2.500000e+00,0.000000e+00,0.000000e+00
50%,2.0,1.0,1.840000e+00,1.0,161.0,162.0,1.0,1.350000e+01,0.000000e+00,5.000000e-01,2.160000e+00,0.000000e+00,1.000000e+00,2.123000e+01,2.500000e+00,7.500000e-01,0.000000e+00
75%,2.0,1.0,3.610000e+00,1.0,233.0,234.0,1.0,2.245000e+01,2.500000e+00,5.000000e-01,4.000000e+00,0.000000e+00,1.000000e+00,3.035000e+01,2.500000e+00,7.500000e-01,0.000000e+00
max,7.0,9.0,3.860884e+05,99.0,265.0,265.0,4.0,1.777500e+03,1.336000e+02,2.214000e+01,5.250000e+02,1.481700e+02,1.000000e+00,1.793370e+03,2.500000e+00,1.250000e+00,6.750000e+00


In [27]:
log_dqv_report_datahub(dqv_results, dataset_name_to_urn)